# TP2 - EA3
Diseñar, simular e implemetar un amplificador de microondas de baja señal para máxima transferencia de energía (adaptación de impedancias de entrada y salida)
Los requisitos son:
- $Z_{in}$ = 50 $\Omega$
- $Z_L$ = 50 $\Omega$
- FR4 $\rightarrow \epsilon_r = 4.5$
- El transistor elegido es BFP 450
 A continuacion se cargan en un arreglo los parámetros S en base a la tensión y corriente de polarización seleccionadas para el transistor

### Se procede a cargar los datos del archivo s2p para poder trabajar con ellos

In [1]:
import os
import numpy as np
import skrf as skrf

#Calcula los parámetros S en una frecuencia dada para cada archivo .s2p en las carpetas especificadas.
def calcular_parametros_s(frecuencia, carpetas):
    s_parameters = {}
    
    # Convertir frecuencia de GHz a Hz
    frecuencia *= 1e9                                  
    
    for carpeta in carpetas:
        archivos_s2p = [f for f in os.listdir(carpeta) if f.endswith('.s2p')]
        for archivo_s2p in archivos_s2p:
            ruta_archivo = os.path.join(carpeta, archivo_s2p)
            red = skrf.network.Network(ruta_archivo)
            
            # Encontrar la frecuencia más cercana en el archivo .s2p
            indice_frecuencia = np.argmin(np.abs(red.f - frecuencia))
                        
            # Acceder a los parámetros S en la frecuencia encontrada
            if archivo_s2p not in s_parameters:
                s_parameters[archivo_s2p] = red.s[indice_frecuencia]
    
    return s_parameters

matplotlib not found while setting up plotting


### Se calcula el determinante para cada conjunto de parametros S

In [2]:
def calcular_delta(parametros_s):
    deltas = {}
    for archivo, parametros in parametros_s.items():
        delta = (parametros[0, 0] * parametros[1, 1]) - (parametros[0, 1] * parametros[1, 0])
        deltas[archivo] = delta

    return deltas

### Se calcula el parametro k de cada conjunto y se filtran los k > 1

In [3]:
def calcular_k(s_parametros, deltas):
    ks = {}
    polarizacion_k_bueno = {}
    for archivo, parametros in s_parametros.items():
        S11 = parametros[0, 0]
        S12 = parametros[0, 1]
        S21 = parametros[1, 0]
        S22 = parametros[1, 1]
        abs_delta = abs(deltas[archivo])
        
        # Calcular el parámetro k
        numerador = (1 - abs(S11)**2 - abs(S22)**2 + abs_delta**2)
        denominador = 2 * abs(S12 * S21)
        k = numerador / denominador
        
        ks[archivo] = k
        if k > 1:
            # Guardar la polarización si k es mayor que 1
            polarizacion_k_bueno[archivo] = {'k': k}
    
    return ks, polarizacion_k_bueno


### Se calculan los coeficientes B1, B2, C1 y C2 y las impedancias Zin y Zout

In [44]:
def calcular_impedancias(Zo, s_parameters, polarizaciones_buenas_k, ic_filter, frecuencia):
    frecuencia *= 1e9  
    for nombre_archivo, polarizacion in polarizaciones_buenas_k.items():
        # Obtener el valor de IC del nombre del archivo
        ic  = float(nombre_archivo.split('_IC_') [1].split('mA')[0])
        vce = float(nombre_archivo.split('_VCE_')[1].split('V')[0])
        impedancias = []

        # Filtro de polarizaciones
        if ic == ic_filter:
            # Obtener los parámetros S de la polarización actual
            parametros_s = s_parameters[nombre_archivo]

            # Asignar los valores de los parámetros S a las variables correspondientes
            S11 = parametros_s[0, 0]
            S12 = parametros_s[0, 1]
            S21 = parametros_s[1, 0]
            S22 = parametros_s[1, 1]
            det = S11 * S22 - S12 * S21                 # Determinate

            # Cálculo de B1, B2, C1, C2
            B1 = 1 + abs(S11)**2 - abs(S22)**2 - abs(det)**2
            B2 = 1 + abs(S22)**2 - abs(S11)**2 - abs(det)**2
            C1 = S11 - det * np.conj(S22)
            C2 = S22 - det * np.conj(S11)
            
            # Cálculo de gamma in
            gamma_in = (B1 - np.sqrt((B1**2) - (4 * (abs(C1)**2)))) / (2 * abs(C1))
            gamma_in_rect = gamma_in * np.cos(np.angle(C1)) + gamma_in * np.sin(np.angle(C1)) * 1j

            # Cálculo de gamma out
            gamma_out = (B2 - np.sqrt((B2**2) - (4 * (abs(C2)**2)))) / (2 * abs(C2))
            gamma_out_rect = gamma_out * np.cos(np.angle(C2)) + gamma_out * np.sin(np.angle(C2)) * 1j

            "Adaptacion Entrada"
            # Cálculo impedancia de entrada serie
            Z_in = Zo * ((1 + gamma_in_rect) / (1 - gamma_in_rect))

            # Cálculo impedancia de entrada paralelo
            Z_in_p = np.real(Z_in) * (1+(np.imag(Z_in)/np.real(Z_in))**2) + 1j*np.imag(Z_in) * (1+(np.real(Z_in)/np.imag(Z_in))**2)

            # Cálculo de C_in adaptacion
            C_in =1/(2*np.pi*frecuencia*np.imag(Z_in_p))

            # Cálculo de transformador lamda entrada
            Z_in_trafo = np.sqrt(Zo*np.real(Z_in_p))

            
            "Adaptacion Salida"
            # Cálculo impedancia de salida serie
            Z_out = Zo * ((1 + gamma_out_rect) / (1 - gamma_out_rect)) 

            # Cálculo de transformador lamda salida
            Z_out_trafo = np.sqrt(Zo*np.real(Z_out))

            # Cálculo de L_out paralelo
            L_out_p = 1j*Z_out_trafo**2/np.imag(Z_out)

            # Cálculo de C_out adaptacion
            C_out =1/(2*np.pi*frecuencia*np.imag(-L_out_p))

               
            # Imprimir los resultados para cada configuración
            print(f"\033[1m\033[31mPolarización: Vce = {vce} [V] - Ic = {ic} [mA]\033[0m")
            print(f"S11: {S11:.4f}, S12: {S12:.4f}, S21: {S21:.4f}, S22: {S22:.4f}")
            print(f"B1 =", "{:.4f}".format(abs(B1)), "{:.4f}".format(np.degrees(np.angle(B1))), "°")
            print(f"B2 =", "{:.4f}".format(abs(B2)), "{:.4f}".format(np.degrees(np.angle(B2))), "°")
            print(f"C1 =", "{:.4f}".format(abs(C1)), "{:.4f}".format(np.degrees(np.angle(C1))), "°")
            print(f"C2 =", "{:.4f}".format(abs(C2)), "{:.4f}".format(np.degrees(np.angle(C2))), "°")
            print("Γᵢₙ      = ","{:.4f}".format(gamma_in), "{:.4f}".format(np.degrees(np.angle(C1))),"°")
            print("Γᵢₙ_rect = ","{:.4f}".format(gamma_in_rect))
            print("Γₒᵤ      = ","{:.4f}".format(gamma_out), "{:.4f}".format(np.degrees(np.angle(C2))),"°")
            print("Γₒᵤ_rect = ","{:.4f}".format(gamma_out_rect))
            print()
            print("ADAPTACION ENTRADA")
            print("Z_in     = ","{:.4f}".format(Z_in))
            print("C_in     = ","{:.4e}".format(C_in), "F") 
            print("Z_in_trafo= ","{:.4f}".format(Z_in_trafo))
            print()
            print("ADAPTACION SALIDA")
            print("Z_out    = ","{:.4f}".format(Z_out))  
            print("C_out    = ","{:.4e}".format(C_out), "F")
            print("Z_out_trafo= ","{:.4f}".format(Z_out_trafo))
            print()

            impedancias.append((Z_in, Z_out))
    
    return impedancias



### Diseño de microtiras

In [40]:
def microtiras(Zo):
    er = 4.3
    A = Zo / 60 * np.sqrt((er + 1) / 2) + ((er - 1) / (er + 1)) * (0.23 + (0.11 / er))
    B = 377 * np.pi / (2 * Zo * np.sqrt(er))

    print("A: ", A, "B: ", B)

    WH = (8 * np.exp(A)) / (np.exp(2 * A) - 2)
    print("WH: ", WH)

    WH = 2 * np.pi * (B - 1 - np.log(2 * B - 1) + (er - 1) / (2 * er) * (np.log(B - 1) + 0.39 * 0.61 / er))
    print("WH: ", WH)

### Ejemplo de uso

In [45]:
frecuencia = 2.2                # GHz
Zo         = 50                 # Impedancia  
carpetas   = ["SPAR/BFP640"]    # Rutas a las carpetas que contienen los archivos .s2p
ic_filter  = 37                 # Corriente para filtrar polaricaciones

s_parameters = calcular_parametros_s(frecuencia, carpetas)
deltas       = calcular_delta(s_parameters)
ks, polarizacion_k_bueno = calcular_k(s_parameters, deltas)
impedancias  = calcular_impedancias(Zo, s_parameters, polarizacion_k_bueno, ic_filter, frecuencia)
microtiras(Zo)


Polarización: Vce = 1.0 [V] - Ic = 37.0 [mA]
S11: -0.5702+0.2956j, S12: 0.0398+0.0461j, S21: 1.6452+3.6953j, S22: 0.0187-0.0191j
B1 = 1.3592 0.0000 °
B2 = 0.5355 0.0000 °
C1 = 0.6484 152.6829 °
C2 = 0.1738 -38.1377 °
Γᵢₙ      =  0.7340 152.6829 °
Γᵢₙ_rect =  -0.6521+0.3368j
Γₒᵤ      =  0.3685 -38.1377 °
Γₒᵤ_rect =  0.2899-0.2276j

ADAPTACION ENTRADA
Z_in     =  8.1128+11.8479j
C_in     =  4.1569e-12 F
Z_in_trafo=  35.6479

ADAPTACION SALIDA
Z_out    =  77.7009-40.9247j
C_out    =  7.6206e-13 F
Z_out_trafo=  62.3301

Polarización: Vce = 1.5 [V] - Ic = 37.0 [mA]
S11: -0.3699+0.1238j, S12: 0.0322+0.0450j, S21: 2.9341+8.1053j, S22: 0.0887-0.1253j
B1 = 0.9515 0.0000 °
B2 = 0.6942 0.0000 °
C1 = 0.4513 164.3307 °
C2 = 0.3128 -44.2674 °
Γᵢₙ      =  0.7206 164.3307 °
Γᵢₙ_rect =  -0.6938+0.1946j
Γₒᵤ      =  0.6285 -44.2674 °
Γₒᵤ_rect =  0.4501-0.4387j

ADAPTACION ENTRADA
Z_in     =  8.2678+6.6953j
C_in     =  4.2794e-12 F
Z_in_trafo=  26.1626

ADAPTACION SALIDA
Z_out    =  61.1190-88.6540j
C_out